In [1]:
import json
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
PATH_bva = "data/annotations/20180415_bva.json"
PATH_cyber = "data/annotations/20180415_cyber_crime.json"
PATH_intellectual = "data/annotations/20180415_intellectual_property.json"
PATH_scotus = "data/annotations/20180415_scotus.json"

PATHS = [PATH_bva, PATH_cyber, PATH_intellectual, PATH_scotus]

In [3]:
import torch
torch.__version__

'1.12.1'

In [4]:
def TF_PN(original, pred):
    original = original.replace("\n"," ").split()
    pred = pred.split()

    # print(original)
    # print(pred)
    
    beggining = [0, 0, 0, 0] #TP, TN, FP, FN
    end = [0, 0, 0, 0]
    i_max = [0, 0, 0, 0]

    if original[0] == pred[0]:
        beggining[0]+=1
        i_max[1]+=1
        end[1]+=1

    else:
        beggining[2]+=1
        i_max[3]+=1

    if original[-1] == pred[-1]:
        end[0]+=1
        i_max[1]+=1
        beggining[1]+=1

    else:
        end[2]+=1
        i_max[3]+=1

    change = False
    for pos in range(len(original)):
        # print(i_max[0], len(pred))
        if i_max[0] == len(pred):
            beggining[3] =0
            break
        
        if original[pos] != pred[0]:
            continue

        else:
            change = True
        # print(beggining, i_max, end)
        i = [0, 0, 0, 0] #TP, TN, FP, FN
        temp_beg = beggining
        temp_end = end

        tip = len(original) if original < pred else pred

        
        for nth, (x, y) in enumerate(zip(range(pos, len(original)), range(len(pred)))):
            if original[x]==pred[y]:
                i[0]+=1
                temp_beg[1]+=1
                temp_end[1]+=1

            else:
                i[2]+=1
                temp_beg[1]+=1
                if nth!=tip:
                    temp_end[3]+=1


        if i[0]>i_max[0]:
            i_max = i
            if temp_beg[3]==0 and pos!=0:
                temp_beg[3]+=1

        elif i[0]==i_max[0] and (i[2] == 0) :
            i_max = i


            beggining = temp_beg
            end = temp_end

    if len(original) < len(pred):
        i_max[2] +=len(pred) - len(original)
    if len(original) > len(pred):
        i_max[3] += len(original) - len(pred)     
        
           

    if not change:
        i_max[2] = len(pred)-1
        

    return beggining, i_max, len(original), len(pred),  end


In [5]:
def accuracy(x):
    if sum(x) == 0:
        return 0
    return (x[0]+x[1])/sum(x)

def recall(x):  #TP, TN, FP, FN
    if x[0] == 0:
        return 0

    return x[0]/(x[0]+x[3])

def precision(x):
    if x[0] == 0:
        return 0

    return x[0]/(x[0]+x[2])

def f1_score(x):
    if x[0] == 0:
        return 0

    return x[0]/(x[0] + 0.5*(x[2]+x[3]))

def specificity(x):
    if x[1] == 0:
        return 0

    return x[1]/(x[1] + x[2])

def acc_recall_precision_f1(x):
    acc, rec, prec, f1, specificity_  = 0, 0, 0, 0, 0
    t = 0
    for i in x:
        acc = acc + accuracy(i)
        rec = rec + recall(i)
        prec = prec + precision(i)
        f1 = f1 + f1_score(i)
        specificity_ = specificity_ + specificity(i)
        

    return acc/len(x), rec/len(x), prec/len(x), f1/len(x), specificity_/len(x)


In [6]:
def raw_str(string):
    return string.replace("\n","\\n").replace("\r","\\r").replace("\t","\\t")

def check_range(pointer_start, pointer_end, ranges):
    intersection_dict = {}
    for i in ranges:
        if pointer_start>=i['start'] and pointer_end>=i['end']:
            intersection_dict[str(i['start'])+'-'+str(i['end'])] = [i['end'] - pointer_start, abs(i['start']-pointer_start)]

        if pointer_start<=i['start'] and pointer_end>=i['end']:
            intersection_dict[str(i['start'])+'-'+str(i['end'])] = [i['end'] - i['start'], abs(i['start']-pointer_start)]
        
        if pointer_start>=i['start'] and pointer_end<=i['end']:
            intersection_dict[str(i['start'])+'-'+str(i['end'])] = [pointer_end - pointer_start, abs(i['start']-pointer_start)]

        if pointer_start<=i['start'] and pointer_end<=i['end']:
            intersection_dict[str(i['start'])+'-'+str(i['end'])] = [pointer_end - i['start'], abs(i['start']-pointer_start)]

    max_intersection_key = None
    max_intersection_len = -1
    max_intersection_start_dist = 1000000
    
    for i in intersection_dict.keys():
        if intersection_dict[i][0]>max_intersection_len or (intersection_dict[i][0]==max_intersection_len and max_intersection_start_dist<intersection_dict[i][1]):
            max_intersection_len =  intersection_dict[i][0]
            max_intersection_key = i
            max_intersection_start_dist = intersection_dict[i][1]

    return max_intersection_key


In [7]:
os.listdir()

['luima_sbd',
 'pysbd_crf_analysis.ipynb',
 'NeuralNetworks',
 'data',
 'punkt.ipynb',
 'dataset_analysis.ipynb',
 '.git',
 '.gitignore',
 'results',
 'testing.ipynb']

In [8]:
def TF_PN(original, pred):
    original = original.replace("\n"," ").split()
    pred = pred.split()

    # print(original)
    # print(pred)
    
    beggining = [0, 0, 0, 0] #TP, TN, FP, FN
    end = [0, 0, 0, 0]
    i_max = [0, 0, 0, 0]

    if original[0] == pred[0]:
        beggining[0]+=1
        i_max[1]+=1
        end[1]+=1

    else:
        beggining[2]+=1
        i_max[3]+=1

    if original[-1] == pred[-1]:
        end[0]+=1
        i_max[1]+=1
        beggining[1]+=1

    else:
        end[2]+=1
        i_max[3]+=1

    change = False
    for pos in range(len(original)):
        # print(i_max[0], len(pred))
        if i_max[0] == len(pred):
            beggining[3] =0
            break
        
        if original[pos] != pred[0]:
            continue

        else:
            change = True
        
        i = [0, 0, 0, 0] #TP, TN, FP, FN
        temp_beg = beggining
        temp_end = end

        tip = len(original) if original < pred else pred

        for nth, (x, y) in enumerate(zip(range(pos, len(original)), range(len(pred)))):
            if original[x]==pred[y]:
                i[0]+=1
                temp_beg[1]+=1
                temp_end[1]+=1

            else:
                i[2]+=1
                temp_beg[1]+=1
                if nth!=tip:
                    temp_end[3]+=1


        if i[0]>i_max[0]:
            i_max = i
            if temp_beg[3]==0 and pos!=0:
                temp_beg[3]+=1

        elif i[0]==i_max[0] and (i[2] == 0) :
            i_max = i


            beggining = temp_beg
            end = temp_end
        

    if not change:
        i_max[2] = len(pred)-2

    return beggining, i_max, len(original), len(pred),  end


In [9]:
import string
import time

characters = list(' '+string.printable.replace(" ",""))
characters.append('UNK')

characters = {x:i for i, x in enumerate(characters)}

window_size = 6
def BIL_model(model, text):
    device='cpu'
    text = text.replace("\r","\n").replace("\t","\n")

    x = []
    offsets = []
    
    for pos, char in enumerate(text):
        if char == ".":
            right = text[pos+1:pos+window_size+1]
            left = text[pos-window_size-1:pos-1]
            if len(right) == 0:
                right = "\n"+ " "*(window_size-1)
            # elif right.replace(".","")[0] == "\n":
            #     continue
            # elif right[0] == "\n":
            #     right = "\n"+ " "*(window_size-1)
            right = right + " "*(window_size-len(right))
            left = " "*(window_size-len(left)) + left

            x.append([left, right])
            offsets.append(pos)


    inputs = []
    for i in x:
        temp = []
        x_ = ''.join(i[0]) + ''.join(i[1])
        for i in x_:
            if i in characters:
                temp.append(characters[i])

            else:
                temp.append(characters['UNK'])
            
        inputs.append(np.asarray(temp))

    inputs = torch.IntTensor(np.asarray(inputs)).to(device)

    pred = model(inputs)

    pred = pred.cpu().detach().numpy().tolist()
    pred_offsets = []

    for each_pred in pred:
        pred_offsets.append(round(each_pred[0]))

    output = []
    start = 0

    for i,j in zip(offsets, pred_offsets):
        if j == 1:
            output.append(text[start:i+1])
            start = i+1

    return output


def BIL_TF_PN(original, pred):
    original = original.replace("\n"," ").split()
    pred = pred.split()

    # print(original)
    # print(pred)
    
    beggining = [0, 0, 0, 0] #TP, TN, FP, FN
    end = [0, 0, 0, 0]
    i_max = [0, 0, 0, 0]

    if original[0] == pred[0]:
        beggining[0]+=1
        i_max[1]+=1
        end[1]+=1

    else:
        beggining[2]+=1
        i_max[3]+=1

    if original[-1] == pred[-1]:
        end[0]+=1
        i_max[1]+=1
        beggining[1]+=1

    else:
        end[2]+=1
        i_max[3]+=1

    change = False
    for pos in range(len(original)):
        # print(i_max[0], len(pred))
        if i_max[0] == len(pred):
            beggining[3] =0
            break
        
        if original[pos] != pred[0]:
            continue

        else:
            change = True
        
        i = [0, 0, 0, 0] #TP, TN, FP, FN
        temp_beg = beggining
        temp_end = end

        tip = len(original) if original < pred else pred

        for nth, (x, y) in enumerate(zip(range(pos, len(original)), range(len(pred)))):
            if original[x]==pred[y]:
                i[0]+=1
                temp_beg[1]+=1
                temp_end[1]+=1

            else:
                i[2]+=1
                temp_beg[1]+=1
                if nth!=tip:
                    temp_end[3]+=1


        if i[0]>i_max[0]:
            i_max = i
            if temp_beg[3]==0 and pos!=0:
                temp_beg[3]+=1

        elif i[0]==i_max[0] and (i[2] == 0) :
            i_max = i


            beggining = temp_beg
            end = temp_end
        

    if not change:
        i_max[2] = len(pred)-2

    return beggining, i_max, len(original), len(pred),  end


def raw_str(string):
    return string.replace("\n","\\n").replace("\r","\\r").replace("\t","\\t")

def check_range(pointer_start, pointer_end, ranges):
    intersection_dict = {}
    for i in ranges:
        if pointer_start>=i['start'] and pointer_end>=i['end']:
            intersection_dict[str(i['start'])+'-'+str(i['end'])] = [i['end'] - pointer_start, abs(i['start']-pointer_start)]

        if pointer_start<=i['start'] and pointer_end>=i['end']:
            intersection_dict[str(i['start'])+'-'+str(i['end'])] = [i['end'] - i['start'], abs(i['start']-pointer_start)]
        
        if pointer_start>=i['start'] and pointer_end<=i['end']:
            intersection_dict[str(i['start'])+'-'+str(i['end'])] = [pointer_end - pointer_start, abs(i['start']-pointer_start)]

        if pointer_start<=i['start'] and pointer_end<=i['end']:
            intersection_dict[str(i['start'])+'-'+str(i['end'])] = [pointer_end - i['start'], abs(i['start']-pointer_start)]

    max_intersection_key = None
    max_intersection_len = -1
    max_intersection_start_dist = 1000000
    
    for i in intersection_dict.keys():
        if intersection_dict[i][0]>max_intersection_len or (intersection_dict[i][0]==max_intersection_len and max_intersection_start_dist<intersection_dict[i][1]):
            max_intersection_len =  intersection_dict[i][0]
            max_intersection_key = i
            max_intersection_start_dist = intersection_dict[i][1]

    return max_intersection_key


def BIL_accuracy(x):
    if sum(x) == 0:
        return 0
    return (x[0]+x[1])/sum(x)

def BIL_recall(x):  #TP, TN, FP, FN
    if x[0] == 0:
        return 0

    return x[0]/(x[0]+x[3])

def BIL_precision(x):
    if x[0] == 0:
        return 0

    return x[0]/(x[0]+x[2])

def BIL_f1_score(x):
    if x[0] == 0:
        return 0

    return x[0]/(x[0] + 0.5*(x[2]+x[3]))

def BIL_specificity(x):
    if x[1] == 0:
        return 0

    return x[1]/(x[1] + x[2])

def BIL_acc_recall_precision_f1(x):
    acc, rec, prec, f1, specificity_  = 0, 0, 0, 0, 0
    t = 0
    for i in x:
        acc = acc + BIL_accuracy(i)
        rec = rec + BIL_recall(i)
        prec = prec + BIL_precision(i)
        f1 = f1 + BIL_f1_score(i)
        specificity_ = specificity_ + BIL_specificity(i)
        

    return acc/len(x), rec/len(x), prec/len(x), f1/len(x), specificity_/len(x)


def BIL_results(model, timer=False):
    results = {}
    total_time = 0

    for path_ in [PATH_cyber]:
        dataset = json.load(open(path_, "r", encoding="utf8"))
        results[path_] = []
        for doc in dataset:
            x = dataset[doc]['text'].replace("\r","\n").replace("\t","\n")
            y = []
            for sentence in dataset[doc]['annotations']:
                y.append(dataset[doc]['text'][sentence['start']:sentence['end']])

            start_time = time.time()
            pred = BIL_model(model, x)

            if timer:
                print("--- Model took %s seconds to predict ---" % (time.time() - start_time))
            
            total_time = total_time + (time.time() - start_time)
            # print(len(y), len(pred))


            pointer = 0
            mappings = []
            filtred_pred = []

            for pred_sentences in pred:
                maped = check_range(pointer, pointer+len(pred_sentences), dataset[doc]['annotations'])
                # if maped != None:
                #     print(raw_str(pred_sentences), raw_str(dataset['59bd4ac35116540935ee6851']['text'][int(maped.split('-')[0]):int(maped.split('-')[1])]), sep=" ====== ", end="\n\n")

                pointer+=len(pred_sentences)

                pred_sentences = pred_sentences.replace("\n"," ")

                if maped is not None :
                    mappings.append(dataset[doc]['text'][int(maped.split('-')[0]):int(maped.split('-')[1])])
                    filtred_pred.append(pred_sentences)
                    results[path_].append(BIL_TF_PN(mappings[-1], filtred_pred[-1]))

    for paths in results:
        B = []
        I = []
        L = []

        for i in results[paths]:
            B.append(i[0])
            I.append(i[1])
            L.append(i[-1])


        print(f"B : {BIL_acc_recall_precision_f1(B)}")
        print(f"I : {BIL_acc_recall_precision_f1(I)}")
        print(f"L : {BIL_acc_recall_precision_f1(L)}\n")

        if timer:
             print(f"--- Model took a total of {total_time} seconds to predict all the documents ---")

        return BIL_acc_recall_precision_f1(B), BIL_acc_recall_precision_f1(I), BIL_acc_recall_precision_f1(L)


In [10]:
import torchinfo
import os
import json
import gensim
import pandas as pd
import matplotlib.pyplot as plt

import pickle
import torch
import time
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


class bilstm_model_attn(nn.Module):
    def __init__(self,hidden_size,n_layer,vocab_size,embedding_size):
        super().__init__()
        self.n_layer = n_layer
        self.hidden_size = hidden_size

        self.embedding_layer = nn.Embedding(vocab_size, embedding_size, padding_idx=0)
        self.lstm_layer = nn.LSTM(embedding_size,hidden_size,n_layer,batch_first = True, bidirectional = True)
        self.dropout = nn.Dropout(0.2)
        self.out = nn.Linear(2*hidden_size,1)


    def attention_net(self, lstm_output, final_state):
        hidden = final_state.view(-1, self.hidden_size * 2, 1)   # hidden : [batch_size, n_hidden * num_directions(=2), 1(=n_layer)]
        attn_weights = torch.bmm(lstm_output, hidden).squeeze(2) # attn_weights : [batch_size, n_step]
        soft_attn_weights = F.softmax(attn_weights, 1)
        # [batch_size, n_hidden * num_directions(=2), n_step] * [batch_size, n_step, 1] = [batch_size, n_hidden * num_directions(=2), 1]
        context = torch.bmm(lstm_output.transpose(1, 2), soft_attn_weights.unsqueeze(2)).squeeze(2)
        return context, soft_attn_weights # context : [batch_size, n_hidden * num_directions(=2)]

    def forward(self,x):
        # hidden = (torch.zeros((self.n_layer,x.shape[0],self.hidden_size)).to(self.device),torch.zeros((self.n_layer,x.shape[0],self.hidden_size)).to(self.device))
        x = self.embedding_layer(x)
        output, (h, final_cell_state) = self.lstm_layer(x)
        final_hidden_state = torch.concat((h[0], h[1]), dim=1)
        attn_output, attention = self.attention_net(output, final_hidden_state)
        x = self.dropout(attn_output)
        x = torch.sigmoid(self.out(x))

        return x

device = "cpu"
model = bilstm_model_attn(256,1,len(characters), 128).to(device)

import torchsummary
torchsummary.summary(model, (12,), dtypes =[torch.LongTensor], device=device)

Layer (type:depth-idx)                   Output Shape              Param #
├─Embedding: 1-1                         [-1, 12, 128]             12,928
├─LSTM: 1-2                              [-1, 12, 512]             790,528
├─Dropout: 1-3                           [-1, 512]                 --
├─Linear: 1-4                            [-1, 1]                   513
Total params: 803,969
Trainable params: 803,969
Non-trainable params: 0
Total mult-adds (M): 0.80
Input size (MB): 0.00
Forward/backward pass size (MB): 0.06
Params size (MB): 3.07
Estimated Total Size (MB): 3.13


Layer (type:depth-idx)                   Output Shape              Param #
├─Embedding: 1-1                         [-1, 12, 128]             12,928
├─LSTM: 1-2                              [-1, 12, 512]             790,528
├─Dropout: 1-3                           [-1, 512]                 --
├─Linear: 1-4                            [-1, 1]                   513
Total params: 803,969
Trainable params: 803,969
Non-trainable params: 0
Total mult-adds (M): 0.80
Input size (MB): 0.00
Forward/backward pass size (MB): 0.06
Params size (MB): 3.07
Estimated Total Size (MB): 3.13

In [11]:
BIL_results(model, timer=True)

--- Model took 0.07420778274536133 seconds to predict ---
--- Model took 0.13851070404052734 seconds to predict ---
--- Model took 0.05753159523010254 seconds to predict ---
--- Model took 0.14737963676452637 seconds to predict ---
--- Model took 0.08783984184265137 seconds to predict ---
--- Model took 0.14676332473754883 seconds to predict ---
--- Model took 0.0848231315612793 seconds to predict ---
--- Model took 0.19522786140441895 seconds to predict ---
--- Model took 0.06178998947143555 seconds to predict ---
--- Model took 0.11026930809020996 seconds to predict ---
--- Model took 0.10772395133972168 seconds to predict ---
--- Model took 0.060060739517211914 seconds to predict ---
--- Model took 0.08044981956481934 seconds to predict ---
--- Model took 0.18143486976623535 seconds to predict ---
--- Model took 0.0296480655670166 seconds to predict ---
--- Model took 0.06705784797668457 seconds to predict ---
--- Model took 0.04059100151062012 seconds to predict ---
--- Model took 

((0.40403849045498147,
  0.16860986547085202,
  0.16860986547085202,
  0.16860986547085202,
  0.41310428010842554),
 (0.36921027831352726,
  0.4026905829596413,
  0.36275096663896084,
  0.37067555702908966,
  0.0089589296253323),
 (0.4185604683442751,
  0.17259758290670318,
  0.1789237668161435,
  0.1731838880217004,
  0.34798312348098775))